A faire : 
- indexer les articles (et indice inversé)
- changer les noms des fichiers json pour qu'ils soient appelés par leur titre

In [ ]:
#pip install wikipedia-api

In [1]:
import wikipediaapi


wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

##  Get data for each category

In [25]:
from random import shuffle
import pandas as pd


def url_categorymembers(categorymembers,list_pages=[], level=0, max_level=3, i=0):
    pages_cat = list(categorymembers.values()) #toutes les pages de la catégorie

    shuffle(pages_cat)
    print(i)

    for c in pages_cat:
        if i==1000: #1000 pages par catégorie
            break

        if c.title[:6]!="Portal" and c.title[:5]!="File:" and c.title[:8]!="Category": # vérifie que c'est bien un article
            i+=1
            #list_urls.append(c.fullurl)
            #list_pages.append(c)
            #list_pages.append(c.title)
            d = {}
            d["titre"] = c.title
            d["contenu"] = c.text
            d["liens"] = list(c.links.keys())
            list_pages.append(d)

        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level: # descend d'un niveau
            i = url_categorymembers(c.categorymembers, list_pages, level=level + 1, max_level=max_level, i=i)[1]

    return list_pages, i

In [ ]:
cat_arts = wiki_wiki.page("Category:The arts") # Arts
arts, _ = url_categorymembers(cat_arts.categorymembers)
pd.DataFrame(arts).to_csv('arts.csv')

In [ ]:
cat_games = wiki_wiki.page("Category:Games") # Games
games, _= url_categorymembers(cat_games.categorymembers)

pd.DataFrame(games).to_csv('games.csv')

In [ ]:
cat_youth = wiki_wiki.page("Category:Youth") # Kids and Teens (not exact)
youth, _ = url_categorymembers(cat_youth.categorymembers)

pd.DataFrame(youth).to_csv('youth.csv')

In [ ]:
cat_reference = wiki_wiki.page("Category:Reference") # Reference
reference, _ = url_categorymembers(cat_reference.categorymembers)

pd.DataFrame(reference).to_csv('reference.csv')

In [ ]:
cat_shopping = wiki_wiki.page("Category:Retailing") # Shopping
shopping, _ = url_categorymembers(cat_shopping.categorymembers)

pd.DataFrame(shopping).to_csv('shopping.csv')

In [ ]:
cat_business = wiki_wiki.page("Category:Business") # Business
business, _ = url_categorymembers(cat_business.categorymembers)

pd.DataFrame(business).to_csv('business.csv')

In [ ]:
cat_health = wiki_wiki.page("Category:Health") # Health
health, _ = url_categorymembers(cat_health.categorymembers)

pd.DataFrame(health).to_csv('health.csv')

In [ ]:
cat_news = wiki_wiki.page("Category:News") # News
news, _ = url_categorymembers(cat_news.categorymembers)

pd.DataFrame(news).to_csv('news.csv')

In [ ]:
cat_geography = wiki_wiki.page("Category:Geography") # Regional (not exact)
geography, _ = url_categorymembers(cat_geography.categorymembers)

pd.DataFrame(geography).to_csv('geography.csv')

In [ ]:
cat_society = wiki_wiki.page("Category:Society") # Society
society, _ = url_categorymembers(cat_society.categorymembers)

pd.DataFrame(society).to_csv('society.csv')

In [ ]:
cat_computers = wiki_wiki.page("Category:Computers") # Computers
computers, _ = url_categorymembers(cat_computers.categorymembers)

df = pd.DataFrame(computers)
df.to_csv('computers.csv')

In [ ]:
cat_home = wiki_wiki.page("Category:Home") # Home
home, _ = url_categorymembers(cat_home.categorymembers)

df = pd.DataFrame(home)
df.to_csv('home.csv')

In [ ]:
cat_recreation = wiki_wiki.page("Category:Recreation") # Recreation
recreation, _ = url_categorymembers(cat_recreation.categorymembers)

pd.DataFrame(recreation).to_csv('recreation.csv')

In [ ]:
cat_science = wiki_wiki.page("Category:Science") # Science
science, _ = url_categorymembers(cat_science.categorymembers)

pd.DataFrame(science).to_csv('science.csv')

In [ ]:
cat_sports = wiki_wiki.page("Category:Sports") # Sports
sports, _ = url_categorymembers(cat_sports.categorymembers)

pd.DataFrame(sports).to_csv('sports.csv')

In [ ]:
cat_world = wiki_wiki.page("Category:World") # World
world, _ = url_categorymembers(cat_world.categorymembers)

pd.DataFrame(world).to_csv('world.csv')

In [ ]:
#pas exécuté

topics = {
    "Arts": arts,
    "Games": games,
    "Youth": youth,
    "Reference": reference, 
    "Shopping": shopping, 
    "Business": business, 
    "Health": health, 
    "News": news,
    "Geography": geography,
    "Society": society,
    "Computers": computers,
    "Home": home,
    "Recreation": recreation,
    "Science": science,
    "Sports": sports,
    "World": world
}


import pickle 

with open('titres_articles_categories.pkl', 'wb') as f:
    pickle.dump(topics, f)
        
#with open('titres_articles_categories.pkl', 'rb') as f:
    #topics = pickle.load(f)

# Indexation des articles

In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init()

df = pd.DataFrame(liste_documents, columns=["docno", "text"])

index_path = "./index"

indexer = pt.IterDictIndexer(index_path)
index_ref = indexer.index(df["text"], df["docno"])

index = pt.IndexFactory.of(index_ref)

# Compute Page Rank vectors (3.2)

## Matrices d'adjacence

Construction de la matrice sans utiliser les fichiers :

In [28]:
import numpy as np


# build the adjacency matrix
def build_adjacency_matrix(df):
    import ast
    num_pages = df.shape[0]
    adjacency_matrix = np.zeros((num_pages, num_pages))
    df["liens"] = df["liens"].apply(ast.literal_eval)
    
    for index, row in df.iterrows():
        for link in row['liens']:
            if link in df['titre'].values:
                j = np.where(df['titre'] == link )[0]
                adjacency_matrix[index][j] += 1

    return adjacency_matrix

In [36]:
matrice_arts = build_adjacency_matrix(pd.read_csv('./Data/arts.csv'))
np.save('./Matrices/matrice_arts.npy', matrice_arts)

matrice_games = build_adjacency_matrix(pd.read_csv('./Data/games.csv'))
np.save('./Matrices/matrice_games.npy', matrice_games)

matrice_youth = build_adjacency_matrix(pd.read_csv('./Data/youth.csv'))
np.save('./Matrices/matrice_youth.npy', matrice_youth)

matrice_reference = build_adjacency_matrix(pd.read_csv('./Data/reference.csv'))
np.save('./Matrices/matrice_reference.npy', matrice_reference)

matrice_shopping = build_adjacency_matrix(pd.read_csv('./Data/shopping.csv'))
np.save('./Matrices/matrice_shopping.npy', matrice_shopping)

matrice_business = build_adjacency_matrix(pd.read_csv('./Data/business.csv'))
np.save('./Matrices/matrice_business.npy', matrice_business)

matrice_health = build_adjacency_matrix(pd.read_csv('./Data/health.csv'))
np.save('./Matrices/matrice_health.npy', matrice_health)

matrice_news = build_adjacency_matrix(pd.read_csv('./Data/news.csv'))
np.save('./Matrices/matrice_news.npy', matrice_news)

matrice_geography = build_adjacency_matrix(pd.read_csv('./Data/geography.csv'))
np.save('./Matrices/matrice_geography.npy', matrice_geography)

matrice_society = build_adjacency_matrix(pd.read_csv('./Data/society.csv'))
np.save('./Matrices/matrice_society.npy', matrice_society)

matrice_computers = build_adjacency_matrix(pd.read_csv('./Data/computers.csv'))
np.save('./Matrices/matrice_computers.npy', matrice_computers)

matrice_home = build_adjacency_matrix(pd.read_csv('./Data/home.csv'))
np.save('./Matrices/matrice_home.npy', matrice_home)

matrice_recreation = build_adjacency_matrix(pd.read_csv('./Data/recreation.csv'))
np.save('./Matrices/matrice_recreation.npy', matrice_recreation)

matrice_science = build_adjacency_matrix(pd.read_csv('./Data/science.csv'))
np.save('./Matrices/matrice_science.npy', matrice_science)

matrice_sports = build_adjacency_matrix(pd.read_csv('./Data/sports.csv'))
np.save('./Matrices/matrice_sports.npy', matrice_sports)

matrice_world = build_adjacency_matrix(pd.read_csv('./Data/world.csv'))
np.save('./Matrices/matrice_world.npy', matrice_world)

Construction de la matrice en utilisant les fichiers :

In [3]:
import pandas
fichier_graphe = pandas.read_csv('enwiki.wikilink_graph.2018-03-01.csv')

# build the adjacency matrix
def build_adjacency_matrix(topic_articles, inversed_index_articles):
    num_pages = len(topic_articles)
    adjacency_matrix = np.zeros((num_pages, num_pages))

    for i,titre in enumerate(topic_articles):
        #links = page_links(pages[i])
        links = fichier_graphe[fichier_graphe['page_title_from']==titre]
        for link in links:
            if link in topic_articles:
                j = topic_articles.index(link)
                adjacency_matrix[i][j] = 1

    return adjacency_matrix/sum(adjacency_matrix)



## Calcul des vecteurs

In [38]:
# compute PageRank scores
def compute_pagerank(M, alpha=0.25):
    # M = adjacency_matrix
    # damping_factor = 1-alpha
    N = M.shape[0]
    teleportation_matrix = np.ones((N, N)) / N # pour page_rank normal, pas topic_sensitive

    M_prime = (1-alpha) * M + alpha * teleportation_matrix

    rank = np.ones(N) / N
    old_rank = np.zeros(N)

    epsilon = 1.0e-6
    max_iterations = 100
    iterations = 0

    while np.sum(np.abs(rank - old_rank)) > epsilon and iterations < max_iterations:
        old_rank = rank.copy()
        rank = np.dot(M_prime, rank)
        iterations += 1

    return rank

In [48]:
# Compute PageRank scores for each topic

Topic_names = ["arts","games","youth","reference", "shopping", "business", "health", "news","geography","society","computers","home","recreation","science","sports","world"]

topic_pagerank = {}
for topic in Topic_names:
    adjacency_matrix = np.load(f'./Matrices/matrice_{topic}.npy')
    pagerank_scores = compute_pagerank(adjacency_matrix)
    topic_pagerank[topic] = pagerank_scores


# Print PageRank scores for each topic
for topic, scores in topic_pagerank.items():
    print(f"Topic: {topic}")
    print(scores[:5])

    

Topic: arts
[3.57792215e+139 9.82148846e+137 3.87579877e+138 1.16556414e+138
 1.53235456e+139]
Topic: games
[7.36900746e+191 7.36900746e+191 8.51526595e+191 7.36900746e+191
 7.36900746e+191]
Topic: youth
[2.17940507e+116 9.09707512e+113 8.86972372e+113 1.05170414e+114
 8.23375819e+113]
Topic: reference
[2.65095748e+81 1.27821056e+82 4.34150774e+81 7.52236481e+80
 8.61124918e+79]
Topic: shopping
[1.90523685e+108 1.72122161e+109 1.72122161e+109 1.57960848e+110
 2.46647545e+109]
Topic: business
[1.06911538e+164 5.65099706e+162 1.52264262e+162 1.10800386e+164
 3.26720829e+165]
Topic: health
[3.01604329e+148 2.61682936e+149 1.64402604e+150 2.38024058e+150
 1.58167267e+150]
Topic: news
[1.89133523e+167 5.95992097e+169 4.82703208e+168 4.15188315e+167
 1.89133523e+167]
Topic: geography
[4.98849099e+205 4.98849099e+205 4.98849099e+205 5.15313879e+205
 4.98849099e+205]
Topic: society
[1.10844416e+106 1.04929651e+106 1.23900060e+106 1.04929651e+106
 1.28248477e+106]
Topic: computers
[1.59373311e+

# Similarity ranking (4.1)

In [13]:
def OSim(t1, t2, n=20):
    # t1, t2 listes
    return len(set(t1[:n])&set(t2[:n]))/n

In [ ]:
import itertools

def KSim(t1, t2):
    # t1, t2 listes
    U = set(t1)|set(t2)
    delta1 = U - set(t1)
    delta2 = U - set(t2)
    t1_prime = t1+list(delta1)
    t2_prime = t1+list(delta2)
    sim = 0
    for u,v in list(itertools.permutations(U, 2)):
        if np.sign(t1_prime.index(u)-t1_prime.index(v))==np.sign(t2_prime.index(u)-t2_prime.index(v)):
            sim+=1
    return sim/(len(U)*(len(U)-1))


# Query-time importance score (3.3)

## Calcul de D

A faire : changer le nom de fichiers json pour qu'ils soient enregistrés sous le nom de leur titre

In [ ]:
import json
from sklearn.feature_extraction.text import CountVectorizer


categories_text = []
for c in topics.keys():
    text=""
    for article in topics[c]:
        with open(f"{article}.json") as json_file:
            json_data = json.load(json_file)
            text+=" "+json_data.text
        categories_text.append(text)

vectorizer = CountVectorizer(stop_words='english')
D = vectorizer.fit_transform(categories_text)
terms_indexes = vectorizer.get_feature_names_out()
    

## Score

In [ ]:
# à changer pour ne calculer que sur les docs qui ont les termes de la query au lieu de tout

def sqd(q, topic_pagerank, D, terms_indexes):
    # D = gives the total number of occurrences of each term in each document, array of shape = classes x terms
    
    p_c = 1/16 # "The quantity P(cj) is not as straightforward. We chose to make it uniform"
    
    indexes = np.in1d(terms_indexes, np.array(q)).nonzero()[0]

    p_q_c = D[indexes]/D.sum(axis=1)

    probas_c_q = p_c * np.product(p_q_c)

    return np.sum(probas_c_q*topic_pagerank,axis=1)

In [ ]:
queries = [ "affirmative action", "alcoholism", "amusement parks", 
            "architecture", "bicycling", "blues", "cheese", 
            "citrus groves", "classical guitar", "computer vision", 
            "cruises", "death valley", "field hockey", 
            "gardening", "graphic design", "gulf war", 
            "hiv", "java", "lipari",
            "lyme disease", "mutual funds", "national parks", 
            "parallel architecture", "recycling cans", "rock climbing", 
            "san francisco", "shakespeare", "stamp collecting", 
            "sushi", "table tennis", "telecommuting", 
            "vintage cars", "volcano", "zen buddhism", "zener"]